In [35]:
import pandas as pd
import typing as tp
import math

In [36]:
class Container(tp.TypedDict):
    load: int
    items: list[int]

In [37]:
Distribution = list[Container]

In [38]:
# Distribution contains the list of containers with the list of items in each container
FittingResult = tp.TypedDict("FittingResult", {
    "distribution": Distribution,
    "containers_num": int,
    "comparison_num": int,
})

In [39]:
def read_data_frame_from_csv(file_path: str, custom_column_names: list = None) -> pd.DataFrame:
  """
  Reads a data frame from a csv file.
  """
  return pd.read_csv(file_path, names=custom_column_names)

def get_data_frame_row(data_frame: pd.DataFrame, row_index: int) -> pd.Series:
  """
  Returns a row of a data frame as a list.
  """
  return data_frame.iloc[row_index]

def get_num_elements_in_row(df: pd.DataFrame, row_index: int) -> int:
  """
  Returns the number of elements in a row of a Pandas DataFrame.
  """
  return len(get_data_frame_row(df, row_index))

def get_num_rows(df: pd.DataFrame) -> int:
  """
  Returns the number of rows in a Pandas DataFrame.
  """
  return df.shape[0]

In [40]:
def merge_sort(array: list[int], ascending_order=True) -> tuple[int, list[int]]:
  """
  Sorts a copy of an array of integers in the specified order.
  """
  comparison_num = 0

  if len(array) > 1:
    # Divide the array into two halves
    mid = len(array) // 2
    left_half = array[:mid]
    right_half = array[mid:]

    # Recursively sort the two halves
    left_data = merge_sort(left_half, ascending_order)
    left_half = left_data[1]
    comparison_num += left_data[0]

    right_data = merge_sort(right_half, ascending_order)
    right_half = right_data[1]
    comparison_num += right_data[0]

    # Merge the sorted halves
    i = j = k = 0
    merged_array = [0] * len(array)

    while i < len(left_half) and j < len(right_half):
      comparison_num += 1

      if (left_half[i] < right_half[j] and ascending_order) or (left_half[i] > right_half[j] and not ascending_order):
        merged_array[k] = left_half[i]
        i += 1
      else:
        merged_array[k] = right_half[j]
        j += 1

      k += 1

    # Copy the remaining elements of left_half, if any
    while i < len(left_half):
      merged_array[k] = left_half[i]
      i += 1
      k += 1

    # Copy the remaining elements of right_half, if any
    while j < len(right_half):
      merged_array[k] = right_half[j]
      j += 1
      k += 1

    return (comparison_num, merged_array)
  else:
    return (comparison_num, array.copy())

In [41]:
def next_fit_algorithm(item_weights: list[int], container_capacity: int) -> FittingResult:
  """
  Runs the next fit algorithm on the data.
  """
  distribution: Distribution = []
  containers_num = 0
  comparison_num = 0

  for weight in item_weights:
    comparison_num += 1

    if len(distribution) > 0 and distribution[-1]["load"] + weight <= container_capacity:
      distribution[-1]["items"].append(weight)
      distribution[-1]["load"] += weight

      continue

    containers_num += 1
    distribution.append(Container(load=weight, items=[weight]))

  return {
      "distribution": distribution,
      "containers_num": containers_num,
      "comparison_num": comparison_num,
  }

In [42]:
def first_fit_algorithm(item_weights: list[int], container_capacity: int) -> FittingResult:
  """
  Runs the first fit algorithm on the data.
  """
  distribution: Distribution = []
  containers_num = 0
  comparison_num = 0

  for weight in item_weights:
    container_found = False
    
    for container in distribution:
      comparison_num += 1

      if container["load"] + weight <= container_capacity:
        container["items"].append(weight)
        container["load"] += weight
        container_found = True

        break

    if not container_found:
      distribution.append(Container(load=weight, items=[weight]))
      containers_num += 1

  return {
      "distribution": distribution,
      "containers_num": containers_num,
      "comparison_num": comparison_num,
  }

In [43]:
def worst_fit_algorithm(item_weights: list[int], container_capacity: int) -> FittingResult:
  """
  Runs the worst fit algorithm on the data
  It finds the container with the most free space and puts the item there
  """
  distribution: Distribution = []
  containers_num = 0
  comparison_num = 0

  for weight in item_weights:
    container_with_min_load_index = -1
    min_load = container_capacity + 1

    if len(distribution) > 0 and distribution[-1]["load"] + weight <= container_capacity:
      distribution[-1]["items"].append(weight)
      distribution[-1]["load"] += weight

      continue

    comparison_num += 1

    for i, container in enumerate(distribution):
      comparison_num += 2
      current_load = container["load"]

      if current_load + weight <= container_capacity and current_load < min_load:
        container_with_min_load_index = i
        min_load = current_load

    min_load = container_capacity + 1

    if container_with_min_load_index == -1:
      distribution.append(Container(load=weight, items=[weight]))
      containers_num += 1

      continue

    container_with_min_load_index = -1
    distribution[container_with_min_load_index]["items"].append(weight)
    distribution[container_with_min_load_index]["load"] += weight

  return {
      "distribution": distribution,
      "containers_num": containers_num,
      "comparison_num": comparison_num,
  }

In [44]:
def best_fit_algorithm(item_weights: list[int], container_capacity: int) -> FittingResult:
  """
  Runs the best fit algorithm on the data
  It finds the most loaded container that has enough space for the item
  """
  distribution: Distribution = []
  containers_num = 0
  comparison_num = 0

  for weight in item_weights:
    container_with_max_load_index = -1
    max_load = -1

    if len(distribution) > 0 and distribution[-1]["load"] + weight <= container_capacity:
      distribution[-1]["items"].append(weight)
      distribution[-1]["load"] += weight

      continue

    comparison_num += 1

    for i, container in enumerate(distribution):
      comparison_num += 2
      current_load = container["load"]

      if current_load + weight <= container_capacity and current_load > max_load:
        container_with_max_load_index = i
        max_load = current_load

    max_load = -1

    if container_with_max_load_index == -1:
      distribution.append(Container(load=weight, items=[weight]))
      containers_num += 1

      continue

    container_with_max_load_index = -1
    distribution[container_with_max_load_index]["items"].append(weight)
    distribution[container_with_max_load_index]["load"] += weight

  return {
      "distribution": distribution,
      "containers_num": containers_num,
      "comparison_num": comparison_num,
  }

In [45]:
def calculate_min_containers_num(item_weights: list[int], container_capacity: int) -> int:
  return math.ceil(sum(item_weights) / container_capacity)

In [46]:
def fit_algos_decorator(algo: tp.Callable[[list[int], int], FittingResult], item_weights: list[int], container_capacity: int, should_sort = False, is_ascending = True) -> FittingResult:
  comparison_num = 0
  
  if should_sort:
    comparison_num, item_weights = merge_sort(item_weights, is_ascending)

  results = algo(item_weights, container_capacity)

  results["comparison_num"] += comparison_num

  return results

In [47]:
def create_table_from(data: dict, row_names: list[str] = None):
    """
    Builds a table from a list of tuples.
    """
    table = pd.DataFrame(data, index=row_names)

    return table

In [ ]:
def transform_distribution_to_dict(data: list[Container], items: list[int]) -> dict:
  new_form = dict()
  items_num = len(items)
  containers_num = len(data)
  containers = [container["items"] for container in data]
  items = items.copy()

  for i in range(1, items_num + 1):
    new_form[f"{i}"] = [""] * containers_num

  for i, container in enumerate(containers):
    for item in container:
      item_index = items.index(item)
      items[item_index] = -1
      new_form[f"{item_index + 1}"][i] = item

  return new_form

In [48]:
def transform_distribution_to_data_frame(data: list[Container], items: list[int]) -> pd.DataFrame:
  new_form = transform_distribution_to_dict(data, items)

  return create_table_from(new_form, row_names=[f"Container {i + 1}" for i in range(len(data))])

In [49]:
def solve(file_path: str, should_sort = False, is_ascending = True):
  identifier = file_path.split('/')[1].split(".")[0]

  if should_sort:
    identifier += "_sorted"

  if should_sort and is_ascending:
    identifier += "_ascending"
  elif should_sort and not is_ascending:
    identifier += "_descending"

  df = read_data_frame_from_csv(file_path)
  rows_num = get_num_rows(df)

  for row_index in range(rows_num):
    row = get_data_frame_row(df, row_index)
    item_weights = [int(weight) for weight in row.to_list()]

    container_capacity = 100

    nfa_result = fit_algos_decorator(next_fit_algorithm, item_weights, container_capacity, should_sort, is_ascending)
    ffa_result = fit_algos_decorator(first_fit_algorithm, item_weights, container_capacity, should_sort, is_ascending)
    bfa_result = fit_algos_decorator(best_fit_algorithm, item_weights, container_capacity, should_sort, is_ascending)
    wfa_result = fit_algos_decorator(worst_fit_algorithm, item_weights, container_capacity, should_sort, is_ascending)

    algorithms_results = {
        "Next Fit": [nfa_result["containers_num"], nfa_result["comparison_num"]],
        "First Fit": [ffa_result["containers_num"], ffa_result["comparison_num"]],
        "Best Fit": [bfa_result["containers_num"], bfa_result["comparison_num"]],
        "Worst Fit": [wfa_result["containers_num"], wfa_result["comparison_num"]],
    }

    distribution_results = {
        "next-fit": nfa_result["distribution"],
        "first-fit": ffa_result["distribution"],
        "best-fit": bfa_result["distribution"],
        "worst-fit": wfa_result["distribution"],
    }

    min_containers_num = calculate_min_containers_num(item_weights, container_capacity)

    algorithms_results["Min Containers Num"] = min_containers_num

    algorithms_results_table = create_table_from(algorithms_results, ["Containers Num", "Comparison Num"])
    algorithms_results_table.to_csv(f"results/{identifier}_{row_index + 1}.csv", index=False)

    for key, value in distribution_results.items():
      distribution = transform_distribution_to_data_frame(value, item_weights)
      distribution.to_csv(f"results/{identifier}_{key}_distribution_{row_index + 1}.csv", index=False)

    distribution = transform_distribution_to_data_frame(nfa_result["distribution"], item_weights)
    distribution.to_csv(f"results/{identifier}_distribution_{row_index + 1}.csv", index=False)

    print(f"(Row #{row_index + 1}) Result:")
    print(algorithms_results_table, "\n")

In [50]:
solve("data/original.csv")

(Row #1) Result:
                Next Fit  First Fit  Best Fit  Worst Fit  Min Containers Num
Containers Num        13         13        13         13                  10
Comparison Num        20         90       169        169                  10 

(Row #2) Result:
                Next Fit  First Fit  Best Fit  Worst Fit  Min Containers Num
Containers Num        13         12        12         12                  11
Comparison Num        20        109       190        190                  11 

(Row #3) Result:
                Next Fit  First Fit  Best Fit  Worst Fit  Min Containers Num
Containers Num        13         12        12         12                  11
Comparison Num        20        111       182        182                  11 



In [51]:
solve("data/original.csv", should_sort=True)

(Row #1) Result:
                Next Fit  First Fit  Best Fit  Worst Fit  Min Containers Num
Containers Num        14         14        14         14                  10
Comparison Num        86        163       262        262                  10 

(Row #2) Result:
                Next Fit  First Fit  Best Fit  Worst Fit  Min Containers Num
Containers Num        15         15        15         15                  11
Comparison Num        83        176       288        288                  11 

(Row #3) Result:
                Next Fit  First Fit  Best Fit  Worst Fit  Min Containers Num
Containers Num        14         14        14         14                  11
Comparison Num        84        165       260        260                  11 



In [52]:
solve("data/original.csv", should_sort=True, is_ascending=False)

(Row #1) Result:
                Next Fit  First Fit  Best Fit  Worst Fit  Min Containers Num
Containers Num        14         13        13         13                  10
Comparison Num        85        167       396        396                  10 

(Row #2) Result:
                Next Fit  First Fit  Best Fit  Worst Fit  Min Containers Num
Containers Num        15         12        12         12                  11
Comparison Num        83        189       382        382                  11 

(Row #3) Result:
                Next Fit  First Fit  Best Fit  Worst Fit  Min Containers Num
Containers Num        14         12        11         11                  11
Comparison Num        85        186       370        370                  11 



In [53]:
solve("data/original-long.csv")

(Row #1) Result:
                Next Fit  First Fit  Best Fit  Worst Fit  Min Containers Num
Containers Num        39         35        34         34                  32
Comparison Num        60        870      1709       1709                  32 



In [54]:
solve("data/original-long.csv", should_sort=True)

(Row #1) Result:
                Next Fit  First Fit  Best Fit  Worst Fit  Min Containers Num
Containers Num        41         41        41         41                  32
Comparison Num       340       1154      1961       1961                  32 



In [55]:
solve("data/original-long.csv", should_sort=True, is_ascending=False)

(Row #1) Result:
                Next Fit  First Fit  Best Fit  Worst Fit  Min Containers Num
Containers Num        41         35        34         34                  32
Comparison Num       338       1292      3159       3159                  32 

